# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 6 2023 8:31AM,254427,19,60017906,"GUSA Granules USA, Inc.",Released
1,Jan 6 2023 8:30AM,254404,10,0086284570,ISDIN Corporation,Released
2,Jan 6 2023 8:30AM,254402,10,SO94465-1,"Senseonics, Incorporated",Released
3,Jan 6 2023 8:30AM,254404,10,0086284562,ISDIN Corporation,Released
4,Jan 6 2023 8:30AM,254404,10,0086284563,ISDIN Corporation,Released
5,Jan 6 2023 8:30AM,254410,10,0086284565,ISDIN Corporation,Released
6,Jan 6 2023 8:30AM,254410,10,0086284568,ISDIN Corporation,Released
7,Jan 6 2023 8:30AM,254410,10,0086284564,ISDIN Corporation,Released
8,Jan 6 2023 8:30AM,254410,10,0086284569,ISDIN Corporation,Released
9,Jan 6 2023 8:30AM,254410,10,0086284566,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,254420,Released,1
43,254424,Released,14
44,254425,Released,1
45,254426,Released,1
46,254427,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254420,NaN,NaN,1.0
254424,NaN,NaN,14.0
254425,NaN,NaN,1.0
254426,NaN,NaN,1.0
254427,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254327,0.0,1.0,0.0
254331,0.0,0.0,1.0
254346,0.0,0.0,1.0
254347,0.0,0.0,1.0
254348,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254327,0,1,0
254331,0,0,1
254346,0,0,1
254347,0,0,1
254348,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254327,0,1,0
1,254331,0,0,1
2,254346,0,0,1
3,254347,0,0,1
4,254348,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254327,,1,
1,254331,,,1
2,254346,,,1
3,254347,,,1
4,254348,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 6 2023 8:31AM,254427,19,"GUSA Granules USA, Inc."
1,Jan 6 2023 8:30AM,254404,10,ISDIN Corporation
2,Jan 6 2023 8:30AM,254402,10,"Senseonics, Incorporated"
5,Jan 6 2023 8:30AM,254410,10,ISDIN Corporation
22,Jan 6 2023 8:30AM,254409,10,ISDIN Corporation
26,Jan 6 2023 8:30AM,254412,10,Eywa Pharma Inc.
51,Jan 6 2023 8:27AM,254426,19,"GUSA Granules USA, Inc."
52,Jan 6 2023 8:17AM,254425,19,"Emersa Waterbox, LLC"
53,Jan 6 2023 7:42AM,254424,19,"AdvaGen Pharma, Ltd"
67,Jan 5 2023 5:06PM,254420,18,Seqirus


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 6 2023 8:31AM,254427,19,"GUSA Granules USA, Inc.",,,1
1,Jan 6 2023 8:30AM,254404,10,ISDIN Corporation,,,19
2,Jan 6 2023 8:30AM,254402,10,"Senseonics, Incorporated",,,1
3,Jan 6 2023 8:30AM,254410,10,ISDIN Corporation,,,16
4,Jan 6 2023 8:30AM,254409,10,ISDIN Corporation,,,3
5,Jan 6 2023 8:30AM,254412,10,Eywa Pharma Inc.,,,11
6,Jan 6 2023 8:27AM,254426,19,"GUSA Granules USA, Inc.",,,1
7,Jan 6 2023 8:17AM,254425,19,"Emersa Waterbox, LLC",,,1
8,Jan 6 2023 7:42AM,254424,19,"AdvaGen Pharma, Ltd",,,14
9,Jan 5 2023 5:06PM,254420,18,Seqirus,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 8:31AM,254427,19,"GUSA Granules USA, Inc.",1,,
1,Jan 6 2023 8:30AM,254404,10,ISDIN Corporation,19,,
2,Jan 6 2023 8:30AM,254402,10,"Senseonics, Incorporated",1,,
3,Jan 6 2023 8:30AM,254410,10,ISDIN Corporation,16,,
4,Jan 6 2023 8:30AM,254409,10,ISDIN Corporation,3,,
5,Jan 6 2023 8:30AM,254412,10,Eywa Pharma Inc.,11,,
6,Jan 6 2023 8:27AM,254426,19,"GUSA Granules USA, Inc.",1,,
7,Jan 6 2023 8:17AM,254425,19,"Emersa Waterbox, LLC",1,,
8,Jan 6 2023 7:42AM,254424,19,"AdvaGen Pharma, Ltd",14,,
9,Jan 5 2023 5:06PM,254420,18,Seqirus,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 8:31AM,254427,19,"GUSA Granules USA, Inc.",1,,
1,Jan 6 2023 8:30AM,254404,10,ISDIN Corporation,19,,
2,Jan 6 2023 8:30AM,254402,10,"Senseonics, Incorporated",1,,
3,Jan 6 2023 8:30AM,254410,10,ISDIN Corporation,16,,
4,Jan 6 2023 8:30AM,254409,10,ISDIN Corporation,3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 8:31AM,254427,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
1,Jan 6 2023 8:30AM,254404,10,ISDIN Corporation,19.0,NaN,NaN
2,Jan 6 2023 8:30AM,254402,10,"Senseonics, Incorporated",1.0,NaN,NaN
3,Jan 6 2023 8:30AM,254410,10,ISDIN Corporation,16.0,NaN,NaN
4,Jan 6 2023 8:30AM,254409,10,ISDIN Corporation,3.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 8:31AM,254427,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
1,Jan 6 2023 8:30AM,254404,10,ISDIN Corporation,19.0,0.0,0.0
2,Jan 6 2023 8:30AM,254402,10,"Senseonics, Incorporated",1.0,0.0,0.0
3,Jan 6 2023 8:30AM,254410,10,ISDIN Corporation,16.0,0.0,0.0
4,Jan 6 2023 8:30AM,254409,10,ISDIN Corporation,3.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3561429,241.0,13.0,10.0
15,508757,5.0,0.0,0.0
16,1272083,6.0,0.0,0.0
18,254420,1.0,0.0,0.0
19,2544019,24.0,0.0,4.0
20,763079,2.0,1.0,0.0
21,2035017,7.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3561429,241.0,13.0,10.0
1,15,508757,5.0,0.0,0.0
2,16,1272083,6.0,0.0,0.0
3,18,254420,1.0,0.0,0.0
4,19,2544019,24.0,0.0,4.0
5,20,763079,2.0,1.0,0.0
6,21,2035017,7.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,241.0,13.0,10.0
1,15,5.0,0.0,0.0
2,16,6.0,0.0,0.0
3,18,1.0,0.0,0.0
4,19,24.0,0.0,4.0
5,20,2.0,1.0,0.0
6,21,7.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,241.0
1,15,Released,5.0
2,16,Released,6.0
3,18,Released,1.0
4,19,Released,24.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21
Status,,,,,,,
Completed,10.0,0.0,0.0,0.0,4.0,0.0,0.0
Executing,13.0,0.0,0.0,0.0,0.0,1.0,1.0
Released,241.0,5.0,6.0,1.0,24.0,2.0,7.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21
0,Completed,10.0,0.0,0.0,0.0,4.0,0.0,0.0
1,Executing,13.0,0.0,0.0,0.0,0.0,1.0,1.0
2,Released,241.0,5.0,6.0,1.0,24.0,2.0,7.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21
0,Completed,10.0,0.0,0.0,0.0,4.0,0.0,0.0
1,Executing,13.0,0.0,0.0,0.0,0.0,1.0,1.0
2,Released,241.0,5.0,6.0,1.0,24.0,2.0,7.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()